# MDF data analysis

In [1]:
from load_data import train, test, BIG, TRAIN, TEST, SOURCE
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

TRAIN (8564, 42)
TEST  (3671, 42)
BIG  (12235, 43)


/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
from visualization_helper import display_groups, display_unique_count
from preprocessing_helper import get_unique_noconst, split_series_values, get_unique_col_values, get_numeric_nonnumeric_cols

In [3]:
print BIG.shape, BIG.columns.values
print train.shape, train.columns.values
print test.shape, test.columns.values


(12235, 43) [u'agrement col' u'date amm annee' u'date declar annee' u'etat commerc'
 u'forme pharma' u'id' u'libelle' u'libelle_ampoule' u'libelle_capsule'
 u'libelle_comprime' u'libelle_film' u'libelle_flacon' u'libelle_gelule'
 u'libelle_pilulier' u'libelle_plaquette' u'libelle_poche'
 u'libelle_sachet' u'libelle_seringue' u'libelle_stylo' u'libelle_tube'
 u'nb_ampoule' u'nb_capsule' u'nb_comprime' u'nb_film' u'nb_flacon'
 u'nb_gelule' u'nb_ml' u'nb_pilulier' u'nb_plaquette' u'nb_poche'
 u'nb_sachet' u'nb_seringue' u'nb_stylo' u'nb_tube' u'prix' 'source'
 u'statut' u'statut admin' u'substances' u'titulaires' u'tx rembours'
 u'type proc' u'voies admin']
(8564, 42) [u'libelle' u'libelle_plaquette' u'libelle_ampoule' u'libelle_flacon'
 u'libelle_tube' u'libelle_stylo' u'libelle_seringue' u'libelle_pilulier'
 u'libelle_sachet' u'libelle_comprime' u'libelle_gelule' u'libelle_film'
 u'libelle_poche' u'libelle_capsule' u'nb_plaquette' u'nb_ampoule'
 u'nb_flacon' u'nb_tube' u'nb_stylo' u'nb_

Remove some useless columns

In [4]:
df__noid_noprix = BIG.drop(['libelle', 'prix', 'id'], axis=1)

Check for dupicate rows and lets look at the columns with only one unique value.

In [5]:
df__noid_noprix_unique_noconst = get_unique_noconst(df__noid_noprix, verbose=True)

-- get_unique_noconst --
- Number of found duplicated rows :  1389
- Remove constant columns :  (10846, 40) -> (10846, 40)
- Dropped const columns : 


Columns description : 

In [6]:
type_groups_func = lambda df : df.columns.to_series().groupby(df.dtypes)
type_groups = type_groups_func(df__noid_noprix_unique_noconst)

In [7]:
display_groups(type_groups)


-- object --

	agrement col
	etat commerc
	forme pharma
	source
	statut
	statut admin
	substances
	titulaires
	tx rembours
	type proc
	voies admin

-- int64 --

	date amm annee
	date declar annee
	libelle_ampoule
	libelle_capsule
	libelle_comprime
	libelle_film
	libelle_flacon
	libelle_gelule
	libelle_pilulier
	libelle_plaquette
	libelle_poche
	libelle_sachet
	libelle_seringue
	libelle_stylo
	libelle_tube
	nb_ampoule
	nb_capsule
	nb_comprime
	nb_film
	nb_flacon
	nb_gelule
	nb_pilulier
	nb_plaquette
	nb_poche
	nb_sachet
	nb_seringue
	nb_stylo
	nb_tube

-- float64 --

	nb_ml


Identify and separate the numeric and non numeric rows.
Report NaN values on numerical columns

In [8]:
num_cols, nonnum_cols = get_numeric_nonnumeric_cols(df__noid_noprix_unique_noconst)
        
nb_isnan = np.sum(df__noid_noprix_unique_noconst[num_cols].apply(np.isnan))

In [9]:
print nb_isnan

date amm annee       0
date declar annee    0
libelle_ampoule      0
libelle_capsule      0
libelle_comprime     0
libelle_film         0
libelle_flacon       0
libelle_gelule       0
libelle_pilulier     0
libelle_plaquette    0
libelle_poche        0
libelle_sachet       0
libelle_seringue     0
libelle_stylo        0
libelle_tube         0
nb_ampoule           0
nb_capsule           0
nb_comprime          0
nb_film              0
nb_flacon            0
nb_gelule            0
nb_pilulier          0
nb_plaquette         0
nb_poche             0
nb_sachet            0
nb_seringue          0
nb_stylo             0
nb_tube              0
nb_ml                0
dtype: int64


Identify non numeric rows.

In [10]:
df__noid_noprix_unique_noconst[nonnum_cols].tail(5)

,agrement col,etat commerc,forme pharma,source,statut,statut admin,substances,titulaires,tx rembours,type proc,voies admin
12230,oui,Déclaration de commercialisation,solution injectable,test,Présentation active,Autorisation active,KÉTOPROFÈNE,PHARMY II,30%,Procédure nationale,intramusculaire
12231,oui,Déclaration de commercialisation,solution injectable,test,Présentation active,Autorisation active,ACIDE GADOTÉRIQUE,GUERBET,65%,Procédure nationale,intra-articulaire
12232,oui,Déclaration de commercialisation,comprimé gastro-résistant(e),test,Présentation active,Autorisation active,"PANTOPRAZOLE SODIQUE SESQUIHYDRATÉ, PANTOPRAZOLE",RANBAXY PHARMACIE GENERIQUES,65%,Procédure décentralisée,orale
12233,oui,Déclaration de commercialisation,solution pour perfusion,test,Présentation active,Autorisation active,"ACIDE ZOLÉDRONIQUE ANHYDRE, ACIDE ZOLÉDRONIQUE...",BIOGARAN,65%,Procédure nationale,intraveineuse
12234,oui,Déclaration de commercialisation,capsule molle,test,Présentation active,Autorisation active,"CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",PHARMA DEVELOPPEMENT,65%,Procédure nationale,orale


In [27]:
train__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TRAIN]
train_unique_col_values = get_unique_col_values(train__noid_noprix_unique_noconst[nonnum_cols])
train_unique_col_values

,0,1,2,3,4,5,6,7,8,9,...,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741
0,agrement col,2,oui,non,,,,,,,...,,,,,,,,,,
1,etat commerc,4,Déclaration de commercialisation,Déclaration d'arrêt de commercialisation,Arrêt de commercialisation (le médicament n'a ...,Déclaration de suspension de commercialisation,,,,,...,,,,,,,,,,
2,forme pharma,190,comprimé pelliculé,comprimé à croquer,gélule à libération prolongée,granulés à libération prolongée,comprimé,capsule molle,comprimé sécable,dispositif,...,,,,,,,,,,
3,source,1,train,,,,,,,,...,,,,,,,,,,
4,statut,2,Présentation active,Présentation abrogée,,,,,,,...,,,,,,,,,,
5,statut admin,4,Autorisation active,Autorisation abrogée,Autorisation archivée,Autorisation retirée,,,,,...,,,,,,,,,,
6,substances,1740,DÉSOGESTREL,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",CHLORHYDRATE DE DILTIAZEM,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",LÉTROZOLE,FLUTAMIDE,BEXAROTÈNE,GLIMÉPIRIDE,...,N-ACÉTYL L-CYSTÉINE,"LISINOPRIL, LISINOPRIL DIHYDRATÉ","ACIDE ALENDRONIQUE, ALENDRONATE MONOSODIQUE MO...",OXYTOCINE,"CONCENTRAT DE CHOLÉCALCIFÉROL, FORME PULVÉRULE...","MANGANÈSE (GLYCÉROPHOSPHATE DE), GLYCÉROPHOSPH...",EXÉNATIDE,"HYDROGÉNOTARTRATE DE RIVASTIGMINE, RIVASTIGMINE","CALCIUM ÉLÉMENT, CALCIUM (CARBONATE DE), CALCI...",ABATACEPT
7,titulaires,325,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),TEVA SANTE,MYLAN SAS,SANOFI AVENTIS FRANCE,EG LABO - LABORATOIRES EUROGENERICS,EISAI (ROYAUME-UNI),SANDOZ,ARROW GENERIQUES,...,,,,,,,,,,
8,tx rembours,5,65%,100%,15%,30%,0%,,,,...,,,,,,,,,,
9,type proc,5,Procédure décentralisée,Procédure de reconnaissance mutuelle,Procédure nationale,Procédure centralisée,Autorisation d'importation parallèle,,,,...,,,,,,,,,,


In [26]:
test__noid_noprix_unique_noconst = df__noid_noprix_unique_noconst[df__noid_noprix_unique_noconst[SOURCE] == TEST]
test_unique_col_values = get_unique_col_values(test__noid_noprix_unique_noconst[nonnum_cols])
test_unique_col_values

,0,1,2,3,4,5,6,7,8,9,...,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143
0,agrement col,2,non,oui,,,,,,,...,,,,,,,,,,
1,etat commerc,3,Déclaration d'arrêt de commercialisation,Déclaration de commercialisation,Arrêt de commercialisation (le médicament n'a ...,,,,,,...,,,,,,,,,,
2,forme pharma,159,comprimé gastro-résistant(e),capsule,comprimé,suspension injectable,émulsion pour perfusion,comprimé pelliculé sécable,solution pour perfusion,comprimé pelliculé,...,,,,,,,,,,
3,source,1,test,,,,,,,,...,,,,,,,,,,
4,statut,2,Présentation active,Présentation abrogée,,,,,,,...,,,,,,,,,,
5,statut admin,5,Autorisation active,Autorisation abrogée,Autorisation archivée,Autorisation retirée,Autorisation suspendue,,,,...,,,,,,,,,,
6,substances,1142,"PANTOPRAZOLE SODIQUE SESQUIHYDRATÉ, PANTOPRAZOLE","NYSTATINE, SULFATE DE POLYMYXINE B, SULFATE DE...",PRAVASTATINE SODIQUE,INSULINE HUMAINE RECOMBINANTE ISOPHANE BIPHASI...,"PHOSPHATE MONOSODIQUE DIHYDRATÉ, ARGININE, MAG...","MÉMANTINE, CHLORHYDRATE DE MÉMANTINE",CHLORURE DE SODIUM,"FUMARATE DE TÉNOFOVIR DISOPROXIL, TÉNOFOVIR DI...",...,"BROMURE D'IPRATROPIUM ANHYDRE, BROMURE D'IPRAT...","HYDROCHLOROTHIAZIDE, RAMIPRIL","PÉRINDOPRIL, PÉRINDOPRIL ARGININE, INDAPAMIDE",CALCIUM (FLUORURE DE),BÉCLOMÉTHASONE (DIPROPIONATE DE),INSULINE HUMAINE RECOMBINANTE ISOPHANE,"CEFPODOXIME PROXÉTIL, CÉFUROXIME",PRUNIER D'AFRIQUE (EXTRAIT DE),"RANITIDINE, CHLORHYDRATE DE RANITIDINE","ACÉTATE D'OCTRÉOTIDE, OCTRÉOTIDE"
7,titulaires,265,TEVA SANTE,INNOTECH INTERNATIONAL,ACTAVIS FRANCE,SANOFI AVENTIS (ALLEMAGNE),B BRAUN MELSUNGEN,BIOGARAN,FRESENIUS KABI FRANCE,GILEAD SCIENCES INTERNATIONAL (ROYAUME-UNI),...,,,,,,,,,,
8,tx rembours,4,65%,30%,15%,100%,,,,,...,,,,,,,,,,
9,type proc,5,Procédure de reconnaissance mutuelle,Procédure nationale,Procédure centralisée,Procédure décentralisée,Autorisation d'importation parallèle,,,,...,,,,,,,,,,


In [19]:
all_unique_col_values = get_unique_col_values(df__noid_noprix_unique_noconst[nonnum_cols])
all_unique_col_values

,0,1,2,3,4,5,6,7,8,9,...,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043
0,agrement col,2,oui,non,,,,,,,...,,,,,,,,,,
1,etat commerc,4,Déclaration de commercialisation,Déclaration d'arrêt de commercialisation,Arrêt de commercialisation (le médicament n'a ...,Déclaration de suspension de commercialisation,,,,,...,,,,,,,,,,
2,forme pharma,218,comprimé pelliculé,comprimé à croquer,gélule à libération prolongée,granulés à libération prolongée,comprimé,capsule molle,comprimé sécable,dispositif,...,,,,,,,,,,
3,source,2,train,test,,,,,,,...,,,,,,,,,,
4,statut,2,Présentation active,Présentation abrogée,,,,,,,...,,,,,,,,,,
5,statut admin,5,Autorisation active,Autorisation abrogée,Autorisation archivée,Autorisation retirée,Autorisation suspendue,,,,...,,,,,,,,,,
6,substances,2042,DÉSOGESTREL,"MONTÉLUKAST ACIDE, MONTÉLUKAST SODIQUE",CHLORHYDRATE DE DILTIAZEM,"ACIDE VALPROÏQUE, VALPROATE DE SODIUM",LÉTROZOLE,FLUTAMIDE,BEXAROTÈNE,GLIMÉPIRIDE,...,TRIMÉBUTINE (MALÉATE DE),"SODIUM (DIHYDROGÉNOPHOSPHATE DE) DIHYDRATÉ, HY...","TRAMADOL, CHLORHYDRATE DE TRAMADOL","ISONIAZIDE, RIFAMPICINE","RILPIVIRINE, FUMARATE DE TÉNOFOVIR DISOPROXIL,...","BROMURE D'IPRATROPIUM ANHYDRE, BROMURE D'IPRAT...",CALCIUM (FLUORURE DE),BÉCLOMÉTHASONE (DIPROPIONATE DE),"RANITIDINE, CHLORHYDRATE DE RANITIDINE","ACÉTATE D'OCTRÉOTIDE, OCTRÉOTIDE"
7,titulaires,353,MAJORELLE LUXEMBOURG SOPARFI (LUXEMBOURG),TEVA SANTE,MYLAN SAS,SANOFI AVENTIS FRANCE,EG LABO - LABORATOIRES EUROGENERICS,EISAI (ROYAUME-UNI),SANDOZ,ARROW GENERIQUES,...,,,,,,,,,,
8,tx rembours,5,65%,100%,15%,30%,0%,,,,...,,,,,,,,,,
9,type proc,5,Procédure décentralisée,Procédure de reconnaissance mutuelle,Procédure nationale,Procédure centralisée,Autorisation d'importation parallèle,,,,...,,,,,,,,,,


In [43]:
index = 1
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Empty DataFrame
Columns: [Value, Count]
Index: []


In [45]:
index = 2
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                                                 Value  Count
113                              pâte pour application      1
84   microsphère et  solution pour usage parentéral...      1
97     poudre et  solvant pour solution pour perfusion      1
145                        suspension buvable à diluer      1
148          suspension par nébuliseur pour inhalation      1
151                       suspension pour instillation      1
153                               système de diffusion      1
156                 émulsion injectable pour perfusion      1
121                                solution buvable ou      1
126                solution filmogène pour application      1
132                 solution pour application à diluer      1
79                   lyophilisat pour usage parentéral      1
24                     comprimé et  comprimé pelliculé      1
26                               comprimé muco-adhésif      1
28                     comprimé orodispersible sécable      1
19      

In [47]:
index = 4
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Empty DataFrame
Columns: [Value, Count]
Index: []


In [48]:
index = 5
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                    Value  Count
4  Autorisation suspendue      1


In [49]:
index = 6
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                                                  Value  Count
1429                                         SIMÉTICONE      3
588    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 5 (POLYOSID...      2
587    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 4 (POLYOSID...      2
593    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 9V (POLYOSI...      2
592    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 9N (POLYOSI...      2
589    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 6B (POLYOSI...      2
591    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 8 (POLYOSID...      2
590    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 7F (POLYOSI...      2
585    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 3 (POLYOSID...      2
586    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 33F (POLYOS...      2
572    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 10A (POLYOS...      2
581    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 2 (POLYOSID...      2
580    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 19F (POLYOS...      2
579    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 19A (POLYOS...      2
578    STREPTOCOCCUS PNEUMONIAE SÉROTYPE 18C (POLYOS...

In [51]:
index = 7
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                                          Value  Count
180        NOVARTIS VACCINES INFLUENZA (ITALIE)      1
195                            PHARMA-RECHERCHE      1
136            LABORATOIRES OMEGA PHARMA FRANCE      1
137                    LABORATOIRES PHARMA 2000      1
143                          LFB-BIOMEDICAMENTS      1
146                         LIPOMED (ALLEMAGNE)      1
150            MAJORELLE LICENSING (LUXEMBOURG)      1
236                                     SOPHARM      1
238                      STATENS SERUM INSTITUT      1
241                              SUNSTAR FRANCE      1
246                     TEVA PHARMA (ALLEMAGNE)      1
251           TMC PHARMA SERVICES (ROYAUME-UNI)      1
252                                   TONIPHARM      1
203       PRO.MED.CS PRAHA (REPUBLIQUE TCHEQUE)      1
212                                     RICHARD      1
226                                   SANTEN OY      1
42                           BIOALLIANCE PHARMA      1
53        

In [52]:
index = 8
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Empty DataFrame
Columns: [Value, Count]
Index: []


In [53]:
index = 9
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

Empty DataFrame
Columns: [Value, Count]
Index: []


In [54]:
index = 10
ll = train_unique_col_values.iloc[index][1]
train_res = split_series_values(train_unique_col_values.iloc[index][2:2+ll])    
train_df = display_unique_count(train_res)

ll = test_unique_col_values.iloc[index][1]
test_res = split_series_values(test_unique_col_values.iloc[index][2:2+ll])    
test_df = display_unique_count(test_res)

print test_df[~test_df['Value'].isin(train_df['Value'])]

                    Value  Count
34               urétrale      1
4   endotrachéobronchique      1


Encode categorical features

In [ ]:
df = df__noid_noprix_unique_noconst[nonnum_cols].drop(['substances', 'voies admin'], axis=1)
out_df = pd.DataFrame(columns=df.columns, index=df.index)
for c in df.columns:
    le = LabelEncoder()    
    le.fit(df[c])
    out_df[c] = le.transform(df[c])
out_df